# Step 0: 
## Load, filter, clean, and aggregate the data at the customer level

In [269]:
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
url = 'https://github.com/amr-khalil/Identify-customer-segmentation/raw/master/store.xls'
df = pd.read_excel(url)
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.570,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.630,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.490,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.160,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.040,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,29002,IN-2014-62366,2014-06-19,2014-06-19,Same Day,KE-16420,Katrina Edelman,Corporate,Kure,Hiroshima,...,OFF-FA-10000746,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",65.100,5,0.0,4.5000,0.010,Medium
51286,35398,US-2014-102288,2014-06-20,2014-06-24,Standard Class,ZC-21910,Zuschuss Carroll,Consumer,Houston,Texas,...,OFF-AP-10002906,Office Supplies,Appliances,Hoover Replacement Belt for Commercial Guardsm...,0.444,1,0.8,-1.1100,0.010,Medium
51287,40470,US-2013-155768,2013-12-02,2013-12-02,Same Day,LB-16795,Laurel Beltran,Home Office,Oxnard,California,...,OFF-EN-10001219,Office Supplies,Envelopes,"#10- 4 1/8"" x 9 1/2"" Security-Tint Envelopes",22.920,3,0.0,11.2308,0.010,High
51288,9596,MX-2012-140767,2012-02-18,2012-02-22,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,...,OFF-BI-10000806,Office Supplies,Binders,"Acco Index Tab, Economy",13.440,2,0.0,2.4000,0.003,Medium


In [161]:
# df =  df[(df.Country=='United States') & (df.Segment=='Consumer')]
# df.head()

# Step 1:
## Create RFM Anaylses for each customers

In [162]:
max_date = df['Order Date'].max().date()
date_diffrence = lambda x : (max_date - x.max().date()).days
No_unique_orders  = lambda x: len(x.unique())

df_RFM = df.groupby('Customer ID').agg({'Order Date': date_diffrence,
                                        'Order ID': No_unique_orders,  
                                        'Sales': sum })

df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
TA-21385,18,25,35668.12080
GT-14710,8,30,34471.89028
TC-20980,27,28,34218.26900
SM-20320,21,21,31125.29496
BW-11110,54,35,30613.61650


In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          51290 non-null  int64         
 1   Order ID        51290 non-null  object        
 2   Order Date      51290 non-null  datetime64[ns]
 3   Ship Date       51290 non-null  datetime64[ns]
 4   Ship Mode       51290 non-null  object        
 5   Customer ID     51290 non-null  object        
 6   Customer Name   51290 non-null  object        
 7   Segment         51290 non-null  object        
 8   City            51290 non-null  object        
 9   State           51290 non-null  object        
 10  Country         51290 non-null  object        
 11  Postal Code     9994 non-null   float64       
 12  Market          51290 non-null  object        
 13  Region          51290 non-null  object        
 14  Product ID      51290 non-null  object        
 15  Ca

In [271]:
df['Order Date'].max().date()

datetime.date(2014, 12, 31)

# Step 2:
## To automate the segmentation we will use 80% quantile for Recency and Monetary

In [273]:
# We will use the 80% quantile for each feature
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM

     Recency  Frequency      Monetary    R    F    M
0.8    124.2       27.0  14184.850056  2.0  2.0  1.2


,Customer ID,Recency,Frequency,Monetary,R,F,M,RMScore
0,TA-21385,18,25,35668.12080,2,1,2,22
1,GT-14710,8,30,34471.89028,2,2,2,22
2,TC-20980,27,28,34218.26900,2,2,2,22
3,SM-20320,21,21,31125.29496,2,1,2,22
4,BW-11110,54,35,30613.61650,2,2,2,22
...,...,...,...,...,...,...,...,...
1585,MP-7470,543,2,58.11000,1,1,1,11
1586,BD-1500,398,3,52.04700,1,1,1,11
1587,RC-9825,741,1,37.44000,1,1,1,11
1588,MG-7890,373,1,19.12800,1,1,1,11


# Step 3:
## Calculate RFM score and sort customers

In [164]:
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM = df_RFM.reset_index()
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'Customer ID': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.head()

,Customer ID,Frequency,Recency,R,M,Monetary
RMScore,,,,,,
22,315,29,24,2,2,18604.0
21,3,23,152,1,2,16592.0
12,957,15,37,2,1,6385.0
11,315,6,286,1,1,1975.0


# Step 4:
## Visualize the Value Matrix and explore some key numbers

In [165]:
# 1) Average Monetary Matrix
df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_M

R,1,2
M,,
2,16592.0,18604.0
1,1975.0,6385.0


In [166]:
# 2) Number of Customer Matrix
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='Customer ID')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_C
# 3) Recency Matrix

R,1,2
M,,
2,3,315
1,315,957


In [274]:
url = "https://miro.medium.com/max/1400/1*zESqrQ1e6Tm5Uwu23PFtGA.png"
from IPython.display import Image
Image(url= "https://miro.medium.com/max/1400/1*zESqrQ1e6Tm5Uwu23PFtGA.png")

In [171]:
Image(url = 'https://miro.medium.com/max/1080/1*wZY7pxdnVgv-34v7gRQ9jw.png')

In [265]:
df = df.groupby("Customer ID").sum()
df = df.drop(["Row ID", "Postal Code"], axis = 1)

KeyError: "['Row ID' 'Postal Code'] not found in axis"

In [267]:
# Imports
from sklearn import mixture
#GMM = mixture.GaussianMixture(n_components=3, covariance_type='full')
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#import sys
#sys.path.append(".")
import renders as rs

In [268]:

# PCA to reduce the fearues into 2 columns 
from sklearn.decomposition import PCA

pca_model = PCA(n_components=2)
pca_model.fit(df)
reduced_data  = pca_model.transform(df)
reduced_data 

# Generate PCA results plot
pca_results = rs.pca_results(df, pca)

ValueError: Shape of passed values is (2, 7), indices imply (2, 5)

In [236]:
# Create range of clusters 
range_n_clusters = list(range(2,11))
print(range_n_clusters)

[2, 3, 4, 5, 6, 7, 8, 9, 10]


In [240]:
# Loop through clusters
for n_clusters in range_n_clusters:
    # TODO: Apply your clustering algorithm of choice to the reduced data 
    clusterer = mixture.GaussianMixture(n_components=n_clusters).fit(reduced_data)

    # TODO: Predict the cluster for each data point
    preds = clusterer.predict(reduced_data)

    # TODO: Find the cluster centers
    centers = clusterer.means_

    # TODO: Predict the cluster for each transformed sample data point
    #sample_preds = clusterer.predict(pca_samples)

    # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
    score = silhouette_score(reduced_data, preds, metric='mahalanobis')
    print ("For n_clusters = {}. The average silhouette_score is : {}".format(n_clusters, score))

For n_clusters = 2. The average silhouette_score is : 0.3197750350552553
For n_clusters = 3. The average silhouette_score is : 0.24376347040566762
For n_clusters = 4. The average silhouette_score is : 0.128191495418849
For n_clusters = 5. The average silhouette_score is : 0.07194969526924093
For n_clusters = 6. The average silhouette_score is : 0.10635510027848845
For n_clusters = 7. The average silhouette_score is : -0.14322692873850665
For n_clusters = 8. The average silhouette_score is : 0.1130970884018949
For n_clusters = 9. The average silhouette_score is : -0.07408320188518962
For n_clusters = 10. The average silhouette_score is : -0.1364068651390016


In [244]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a mixture of Gaussians with EM
        gmm = mixture.GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(reduced_data)
        bic.append(gmm.bic(reduced_data))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

# KNN Implementation



In [247]:
# Loop through clusters
for n_clusters in range_n_clusters:
    # TODO: Apply your clustering algorithm of choice to the reduced data 
    clusterer = KMeans(n_clusters=n_clusters).fit(reduced_data)

    # TODO: Predict the cluster for each data point
    preds = clusterer.predict(reduced_data)

    # TODO: Find the cluster centers
    centers = clusterer.cluster_centers_

    # TODO: Predict the cluster for each transformed sample data point
    #sample_preds = clusterer.predict(pca_samples)

    # TODO: Calculate the mean silhouette coefficient for the number of clusters chosen
    score = silhouette_score(reduced_data, preds, metric='euclidean')
    print("For n_clusters = {}. The average silhouette_score is : {}".format(n_clusters, score))

For n_clusters = 2. The average silhouette_score is : 0.5620475730139105
For n_clusters = 3. The average silhouette_score is : 0.47225976727039787
For n_clusters = 4. The average silhouette_score is : 0.44263146310989315
For n_clusters = 5. The average silhouette_score is : 0.44588384914420714
For n_clusters = 6. The average silhouette_score is : 0.4273928490073594
For n_clusters = 7. The average silhouette_score is : 0.4228733447174936
For n_clusters = 8. The average silhouette_score is : 0.42517264194379634
For n_clusters = 9. The average silhouette_score is : 0.4171864810337272
For n_clusters = 10. The average silhouette_score is : 0.42378485528047005


In [258]:
# Extra code because we ran a loop on top and this resets to what we want
clusterer = mixture.GaussianMixture(n_components=2).fit(reduced_data)
preds = clusterer.predict(reduced_data)
centers = clusterer.means_
#sample_preds = clusterer.predict(pca_samples)